Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Do the [Plotly Dash](https://dash.plot.ly/) Tutorial, Parts 1 & 2.
- [ ] Add your own stretch goal(s) !

In [1]:
# If you're in Colab...
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install required python packages:
    # pandas-profiling, version >= 2.0
    # plotly, version >= 4.0
    !pip install --upgrade pandas-profiling plotly
    
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Change into directory for module
    os.chdir('module1')

     |████████████████████████████████| 133kB 4.7MB/s 
     |████████████████████████████████| 7.1MB 29.1MB/s 
     |████████████████████████████████| 614kB 34.3MB/s 
     |████████████████████████████████| 225kB 33.0MB/s 
     |████████████████████████████████| 768kB 55.6MB/s 
     |████████████████████████████████| 194kB 59.0MB/s 
     |████████████████████████████████| 51kB 22.1MB/s 
     |████████████████████████████████| 51kB 21.5MB/s 
     |████████████████████████████████| 737kB 38.3MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-2.3.0-py2.py3-none-any.whl size=145035 sha256=a9d236aa32d4f3944b962041082b8f9076ffdce5418af0ae3ee189234e68804e
  Stored in directory: /root/.cache/pip/wheels/ce/c7/f1/dbfef4848ebb048cb1d4a22d1ed0c62d8ff2523747235e19fe
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp36-none-any.whl size=27084 sha256=823a43062ae720bd446f8679895cf8dac8a1438e099a8efd8d2ccbe8b8179f38
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d7

In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
# import numpy, pandas.
import numpy as np
import pandas as pd

In [0]:
# Read New York City apartment rental listing data
df = pd.read_csv('../data/renthop-nyc.csv')
assert df.shape == (49352, 34)
# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [0]:
# show the data frame with headers.
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# use datetaime on 'created' column.
df['created'] = pd.to_datetime(df['created'])
# set index on 'created'.
df = df.set_index(df['created'])
# sort the inex 'created' column.
df = df.sort_index()

In [6]:
# created the train data 4/1 to 5/31.
dftrain = df['2016-04-01':'2016-05-31']
# create the test data 6/1 to 6/30.
dftest  = df['2016-06-01':'2016-06-30']
# show the shape of both data frames.
print("Train Dataset 'April 2016 + May 2016':",dftrain.shape)
print("Test Dataset'June 2016':",dftest.shape)

Train Dataset 'April 2016 + May 2016': (31844, 34)
Test Dataset'June 2016': (16973, 34)


In [7]:
# show the train data frame with headers.
dftrain.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
created,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-04-01 22:12:41,1.0,1,2016-04-01 22:12:41,X-LARGE Flex 1BR Loft! ~~ PRIME Greenwich Vill...,Astor Place,40.7302,-73.9924,3195,1 Astor Place,high,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-01 22:56:00,1.0,0,2016-04-01 22:56:00,"This Enormous Studio Features: Harwood Floors,...",East 54th Street,40.7576,-73.9677,2000,230 East 54th Street,medium,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-01 22:57:15,2.0,3,2016-04-01 22:57:15,--- East 31st St & Lexington Avenue --- This S...,East 31st St & Lexington Avenue,40.7388,-73.9851,5850,105 Lexington Avenue,high,1,1,0,1,1,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
2016-04-01 23:26:07,1.0,1,2016-04-01 23:26:07,Reduced Fee!! Priced To Rent!\rLarge Newly Upd...,West End Ave,40.7939,-73.9738,2745,700 West End Ave,medium,1,1,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
2016-04-02 00:48:13,1.0,1,2016-04-02 00:48:13,Phenomenal deal of the century!! This spacious...,E 88th street,40.7784,-73.9491,2400,401 E 88th street,medium,1,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
# show the teast data frame with headers.
dftest.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
created,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-06-01 01:10:37,1.0,1,2016-06-01 01:10:37,beautiful one bedroom in Williamsburg fit for 1,Montrose Avenue,40.7071,-73.9475,2150,61 Montrose Avenue,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-06-01 01:11:06,1.0,1,2016-06-01 01:11:06,"Welcome home to charming, tree-lined Quincy St...",Quincy Street,40.6877,-73.9450,2795,387 Quincy Street,low,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-06-01 01:11:12,1.0,2,2016-06-01 01:11:12,A beautiful 2 bedroom Apartment in the East Vi...,Saint Marks Place,40.7291,-73.9895,3495,8 Saint Marks Place,low,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-06-01 01:11:52,1.0,0,2016-06-01 01:11:52,"Nice studio, hardwood floors, exposed brick, d...",East 2nd Street,40.7213,-73.9822,2100,237 East 2nd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-06-01 01:12:22,1.0,2,2016-06-01 01:12:22,Great bright and spacious 2 bedrooms two bathr...,East 118th Street,40.7986,-73.9372,2900,249 East 118th Street,low,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Bedrooms and Bathrooms Features.

### Linear Regression.

In [0]:
# import LinearRegression from sklearn.linear_model.
from sklearn.linear_model import LinearRegression

In [10]:
# label and create LinearRegression() model for train data.
model = LinearRegression()
# set all the features from the data set we want to use.
features = ['bedrooms', 'bathrooms']
# set the target to 'price'.
target = 'price'
# set the 'features' data to X.
X = dftrain[features]
# set the 'target' data to Y.
y = dftrain[target]
# .fit the X and Y data to the model.
model.fit(X, y)
# label and create them model.predict(X).
y_pred = model.predict(X)

####

# label and create LinearRegression() model for test data.
model1 = LinearRegression()
# set all the features from the data set we want to use.
features1 = ['bedrooms', 'bathrooms']
# set the target to 'price'.
target1 = 'price'
# set the 'features' data to X1.
X1 = dftest[features1]
# set the 'target' data to Y1.
y1 = dftest[target1]
# .fit the X1 and Y1 data to the model.
model1.fit(X1, y1)
# label and create them model.predict(X1).
y_pred1 = model1.predict(X1)
# show the results.
print("(train data) 2 bedroom 2 bathrooms 'predicted' price =", model.predict(np.array([[2, 2]])))
print("(train data) 2 bedroom 1 bathrooms 'predicted' price =", model.predict(np.array([[2, 1]])))
print("(train data) intercept =", model.intercept_)
print("(train data) coefficient =", model.coef_)
print("(test data) 2 bedroom 2 bathrooms 'predicted' price =", model1.predict(np.array([[2, 2]])))
print("(test data) 2 bedroom 1 bathrooms 'predicted' price =", model1.predict(np.array([[2, 1]])))
print("(test data) intercept =", model1.intercept_)
print("(test data) coefficient =", model1.coef_)

(train data) 2 bedroom 2 bathrooms 'predicted' price = [5409.5887146]
(train data) 2 bedroom 1 bathrooms 'predicted' price = [3336.97859821]
(train data) intercept = 485.7186900232223
(train data) coefficient = [ 389.3248959  2072.61011639]
(test data) 2 bedroom 2 bathrooms 'predicted' price = [5479.43665375]
(test data) 2 bedroom 1 bathrooms 'predicted' price = [3329.18928321]
(test data) intercept = 425.08559409771306
(test data) coefficient = [ 376.92815928 2150.24737055]


In [12]:
import plotly.express as px
px.scatter(dftrain, x='bedrooms', y='bathrooms', color='price')

In [13]:
import plotly.express as px
px.scatter(dftest, x='bedrooms', y='bathrooms', color='price')

### RMSE, MAE, and $R^2$.

In [0]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import mean_absolute_error, r2_score

In [0]:
rms = sqrt(mean_squared_error(y, y_pred))
mae = mean_absolute_error(y, y_pred)
r2 = r2_score(y, y_pred)
rms1 = sqrt(mean_squared_error(y1, y_pred1))
mae1 = mean_absolute_error(y1, y_pred1)
r21 = r2_score(y1, y_pred1)
print("(train data) root mean square error =", rms)
print("(train data)  mean absolute error =", mae)
print("(train data)  R2 score =", r2)
print("(test data) root mean square error =", rms1)
print("(test data)  mean absolute error =", mae1)
print("(test data)  R2 score =", r21)

(train data) root mean square error = 1232.0225917223484
(train data)  mean absolute error = 818.5310213271715
(train data)  R2 score = 0.5111543084316608
(test data) root mean square error = 1219.2441640676209
(test data)  mean absolute error = 828.1524829496833
(test data)  R2 score = 0.5217032949584466


## Bathrooms and Outdoor Space.

### Linear Regression.

In [0]:
# label and create LinearRegression() model for train data.
model = LinearRegression()
# set all the features from the data set we want to use.
features = ['bathrooms', 'outdoor_space']
# set the target to 'price'.
target = 'price'
# set the 'features' data to X.
X = dftrain[features]
# set the 'target' data to Y.
y = dftrain[target]
# .fit the X and Y data to the model.
model.fit(X, y)
# label and create them model.predict(X).
y_pred = model.predict(X)

####

# label and create LinearRegression() model for test data.
model1 = LinearRegression()
# set all the features from the data set we want to use.
features1 = ['bathrooms', 'outdoor_space']
# set the target to 'price'.
target1 = 'price'
# set the 'features' data to X1.
X1 = dftest[features1]
# set the 'target' data to Y1.
y1 = dftest[target1]
# .fit the X1 and Y1 data to the model.
model1.fit(X1, y1)
# label and create them model.predict(X1).
y_pred1 = model1.predict(X1)
# show the results.
print("(train data) 2 bathrooms with outdoor space 'predicted' price =", model.predict(np.array([[2, 1]])))
print("(train data) 2 bathrooms no outdoor space 'predicted' price =", model.predict(np.array([[2, 0]])))
print("(train data) intercept =", model.intercept_)
print("(train data) coefficient =", model.coef_)
print("(test data) 2 bathrooms with outdoor space 'predicted' price =", model1.predict(np.array([[2, 1]])))
print("(test data) 2 bathrooms no outdoor space 'predicted' price =", model1.predict(np.array([[2, 0]])))
print("(test data) intercept =", model1.intercept_)
print("(test data) coefficient =", model1.coef_)

(train data) 2 bathrooms with outdoor space 'predicted' price = [5781.14842185]
(train data) 2 bathrooms no outdoor space 'predicted' price = [5557.2243596]
(train data) intercept = 499.00178687940434
(train data) coefficient = [2529.11128636  223.92406225]
(test data) 2 bathrooms with outdoor space 'predicted' price = [5858.69454263]
(test data) 2 bathrooms no outdoor space 'predicted' price = [5634.70460131]
(test data) intercept = 455.8156839112198
(test data) coefficient = [2589.4444587   223.98994131]


In [14]:
import plotly.express as px
px.scatter(dftrain, x='bathrooms', y='outdoor_space', color='price')

In [15]:
import plotly.express as px
px.scatter(dftest, x='bathrooms', y='outdoor_space', color='price')

### RMSE, MAE, and $R^2$.

In [0]:
rms = sqrt(mean_squared_error(y, y_pred))
mae = mean_absolute_error(y, y_pred)
r2 = r2_score(y, y_pred)
rms1 = sqrt(mean_squared_error(y1, y_pred1))
mae1 = mean_absolute_error(y1, y_pred1)
r21 = r2_score(y1, y_pred1)
print("(train data) root mean square error =", rms)
print("(train data)  mean absolute error =", mae)
print("(train data)  R2 score =", r2)
print("(test data) root mean square error =", rms1)
print("(test data)  mean absolute error =", mae1)
print("(test data)  R2 score =", r21)

(train data) root mean square error = 1282.8381505969444
(train data)  mean absolute error = 884.1048784575621
(train data)  R2 score = 0.4699971771127297
(test data) root mean square error = 1268.154572190475
(test data)  mean absolute error = 889.5578798116944
(test data)  R2 score = 0.4825595192664889


## Longitude and Latitude Space.

### Linear Regression.

In [0]:
# label and create LinearRegression() model for train data.
model = LinearRegression()
# set all the features from the data set we want to use.
features = ['longitude', 'latitude']
# set the target to 'price'.
target = 'price'
# set the 'features' data to X.
X = dftrain[features]
# set the 'target' data to Y.
y = dftrain[target]
# .fit the X and Y data to the model.
model.fit(X, y)
# label and create them model.predict(X).
y_pred = model.predict(X)

####

# label and create LinearRegression() model for test data.
model1 = LinearRegression()
# set all the features from the data set we want to use.
features1 = ['longitude', 'latitude']
# set the target to 'price'.
target1 = 'price'
# set the 'features' data to X1.
X1 = dftest[features1]
# set the 'target' data to Y1.
y1 = dftest[target1]
# .fit the X1 and Y1 data to the model.
model1.fit(X1, y1)
# label and create them model.predict(X1).
y_pred1 = model1.predict(X1)
# show the results.
print("(train data) 2 bathrooms with outdoor space 'predicted' price =", model.predict(np.array([[2, 1]])))
print("(train data) 2 bathrooms no outdoor space 'predicted' price =", model.predict(np.array([[2, 0]])))
print("(train data) intercept =", model.intercept_)
print("(train data) coefficient =", model.coef_)
print("(test data) 2 bathrooms with outdoor space 'predicted' price =", model1.predict(np.array([[2, 1]])))
print("(test data) 2 bathrooms no outdoor space 'predicted' price =", model1.predict(np.array([[2, 0]])))
print("(test data) intercept =", model1.intercept_)
print("(test data) coefficient =", model1.coef_)

(train data) 2 bathrooms with outdoor space 'predicted' price = [-1316155.20627438]
(train data) 2 bathrooms no outdoor space 'predicted' price = [-1318363.39599334]
(train data) intercept = -1285931.985165547
(train data) coefficient = [-16215.70541389   2208.18971896]
(test data) 2 bathrooms with outdoor space 'predicted' price = [-1364009.41961739]
(test data) 2 bathrooms no outdoor space 'predicted' price = [-1366620.11739489]
(test data) intercept = -1333349.7055827554
(test data) coefficient = [-16635.20590607   2610.6977775 ]


In [16]:
import plotly.express as px
px.scatter(dftrain, x='longitude', y='latitude', color='price')

In [17]:
import plotly.express as px
px.scatter(dftest, x='longitude', y='latitude', color='price')

### RMSE, MAE, and $R^2$.

In [0]:
rms = sqrt(mean_squared_error(y, y_pred))
mae = mean_absolute_error(y, y_pred)
r2 = r2_score(y, y_pred)
rms1 = sqrt(mean_squared_error(y1, y_pred1))
mae1 = mean_absolute_error(y1, y_pred1)
r21 = r2_score(y1, y_pred1)
print("(train data) root mean square error =", rms)
print("(train data)  mean absolute error =", mae)
print("(train data)  R2 score =", r2)
print("(test data) root mean square error =", rms1)
print("(test data)  mean absolute error =", mae1)
print("(test data)  R2 score =", r21)

(train data) root mean square error = 1704.2079552598761
(train data)  mean absolute error = 1147.1493278231212
(train data)  R2 score = 0.06463820907125295
(test data) root mean square error = 1702.9223680796963
(test data)  mean absolute error = 1145.2534320943037
(test data)  R2 score = 0.06694827937588999


## Elevator and Common Outdoor Space.

### linear Regression.

In [0]:
# label and create LinearRegression() model for train data.
model = LinearRegression()
# set all the features from the data set we want to use.
features = ['elevator', 'common_outdoor_space']
# set the target to 'price'.
target = 'price'
# set the 'features' data to X.
X = dftrain[features]
# set the 'target' data to Y.
y = dftrain[target]
# .fit the X and Y data to the model.
model.fit(X, y)
# label and create them model.predict(X).
y_pred = model.predict(X)

####

# label and create LinearRegression() model for test data.
model1 = LinearRegression()
# set all the features from the data set we want to use.
features1 = ['elevator', 'common_outdoor_space']
# set the target to 'price'.
target1 = 'price'
# set the 'features' data to X1.
X1 = dftest[features1]
# set the 'target' data to Y1.
y1 = dftest[target1]
# .fit the X1 and Y1 data to the model.
model1.fit(X1, y1)
# label and create them model.predict(X1).
y_pred1 = model1.predict(X1)
# show the results.
print("(train data) 2 bathrooms with outdoor space 'predicted' price =", model.predict(np.array([[2, 1]])))
print("(train data) 2 bathrooms no outdoor space 'predicted' price =", model.predict(np.array([[2, 0]])))
print("(train data) intercept =", model.intercept_)
print("(train data) coefficient =", model.coef_)
print("(test data) 2 bathrooms with outdoor space 'predicted' price =", model1.predict(np.array([[2, 1]])))
print("(test data) 2 bathrooms no outdoor space 'predicted' price =", model1.predict(np.array([[2, 0]])))
print("(test data) intercept =", model1.intercept_)
print("(test data) coefficient =", model1.coef_)

(train data) 2 bathrooms with outdoor space 'predicted' price = [4453.76931925]
(train data) 2 bathrooms no outdoor space 'predicted' price = [4655.09311728]
(train data) intercept = 3193.929935303338
(train data) coefficient = [ 730.58159099 -201.32379803]
(test data) 2 bathrooms with outdoor space 'predicted' price = [4732.21365166]
(test data) 2 bathrooms no outdoor space 'predicted' price = [4697.25442682]
(test data) intercept = 3201.706689631647
(test data) coefficient = [747.7738686   34.95922484]


In [0]:
import plotly.express as px
px.scatter(dftrain, x='elevator', y='latitude', color='price')

### RMSE, MAE, and $R^2$.

In [0]:
rms = sqrt(mean_squared_error(y, y_pred))
mae = mean_absolute_error(y, y_pred)
r2 = r2_score(y, y_pred)
rms1 = sqrt(mean_squared_error(y1, y_pred1))
mae1 = mean_absolute_error(y1, y_pred1)
r21 = r2_score(y1, y_pred1)
print("(train data) root mean square error =", rms)
print("(train data)  mean absolute error =", mae)
print("(train data)  R2 score =", r2)
print("(test data) root mean square error =", rms1)
print("(test data)  mean absolute error =", mae1)
print("(test data)  R2 score =", r21)

(train data) root mean square error = 1724.5217237580869
(train data)  mean absolute error = 1174.3449736441487
(train data)  R2 score = 0.042206715360120994
(test data) root mean square error = 1722.7758600514044
(test data)  mean absolute error = 1170.2212883958011
(test data)  R2 score = 0.0450655229825635
